# Investigating Signs of High "Impact" Power Outages

**Name(s)**: Turki Alrasheed

**Website Link**: https://turkial2005.github.io/Power_Outage_Impact_Analysis/

In [213]:
import pandas as pd
import numpy as np

import plotly.express as px
pd.options.plotting.backend = 'plotly'
import plotly.io as pio
pio.renderers.default ="vscode"
import plotly.figure_factory as ff


## Step 1: Introduction

Research Question: What are the signs indicating that a power outage has a big “impact” on people’s lives? “Impact” is a special metric created from a combination of number of customers lost and duration of power outage.


## Step 2: Data Cleaning and Exploratory Data Analysis

In [214]:
df = pd.read_csv('power_outage.csv') # used google sheets to get the whole df as a csv
df = df.drop(0,axis=0) # first row is not an observation so need to remove it
# getting columns we are interested in
outage_df = df[['U.S._STATE','POSTAL.CODE', 'NERC.REGION','CAUSE.CATEGORY','CLIMATE.REGION',
            'ANOMALY.LEVEL','CLIMATE.CATEGORY','OUTAGE.DURATION','CUSTOMERS.AFFECTED', 
            'TOTAL.CUSTOMERS', 'TOTAL.PRICE', 'TOTAL.SALES', 'TOTAL.REALGSP','UTIL.REALGSP','UTIL.CONTRI',
            'PC.REALGSP.REL','POPULATION','POPDEN_URBAN', 'MONTH','YEAR','OUTAGE.START.DATE',
            'OUTAGE.START.TIME','OUTAGE.RESTORATION.DATE','OUTAGE.RESTORATION.TIME']]

outage_df

,U.S._STATE,POSTAL.CODE,NERC.REGION,CAUSE.CATEGORY,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.DURATION,CUSTOMERS.AFFECTED,TOTAL.CUSTOMERS,TOTAL.PRICE,TOTAL.SALES,TOTAL.REALGSP,UTIL.REALGSP,UTIL.CONTRI,PC.REALGSP.REL,POPULATION,POPDEN_URBAN,MONTH,YEAR,OUTAGE.START.DATE,OUTAGE.START.TIME,OUTAGE.RESTORATION.DATE,OUTAGE.RESTORATION.TIME
1,Minnesota,MN,MRO,severe weather,East North Central,-0.3,normal,3060,70000.0,2595696.0,9.28,6562520,274182,4802,1.751391412,1.077375699,5348119.0,2279,7.0,2011.0,"Friday, July 1, 2011",5:00:00 PM,"Sunday, July 3, 2011",8:00:00 PM
2,Minnesota,MN,MRO,intentional attack,East North Central,-0.1,normal,1,NaN,2640737.0,9.28,5284231,291955,5226,1.790001884,1.089792426,5457125.0,2279,5.0,2014.0,"Sunday, May 11, 2014",6:38:00 PM,"Sunday, May 11, 2014",6:39:00 PM
3,Minnesota,MN,MRO,severe weather,East North Central,-1.5,cold,3000,70000.0,2586905.0,8.15,5222116,267895,4571,1.706265514,1.066825978,5310903.0,2279,10.0,2010.0,"Tuesday, October 26, 2010",8:00:00 PM,"Thursday, October 28, 2010",10:00:00 PM
4,Minnesota,MN,MRO,severe weather,East North Central,-0.1,normal,2550,68200.0,2606813.0,9.19,5787064,277627,5364,1.932088738,1.071476036,5380443.0,2279,6.0,2012.0,"Tuesday, June 19, 2012",4:30:00 AM,"Wednesday, June 20, 2012",11:00:00 PM
5,Minnesota,MN,MRO,severe weather,East North Central,1.2,warm,1740,250000.0,2673531.0,10.43,5970339,292023,4873,1.668704177,1.092027125,5489594.0,2279,7.0,2015.0,"Saturday, July 18, 2015",2:00:00 AM,"Sunday, July 19, 2015",7:00:00 AM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1530,North Dakota,ND,MRO,public appeal,West North Central,-0.9,cold,720,34500.0,394394.0,7.56,1313678,39067,934,2.390764584,1.19808347,685326.0,2192.2,12.0,2011.0,"Tuesday, December 6, 2011",8:00:00 AM,"Tuesday, December 6, 2011",8:00:00 PM
1531,North Dakota,ND,MRO,fuel supply emergency,West North Central,NaN,NaN,NaN,NaN,366037.0,NaN,NaN,27868,1019,3.656523611,0.877404977,649422.0,2192.2,NaN,2006.0,NaN,NaN,NaN,NaN
1532,South Dakota,SD,RFC,islanding,West North Central,0.5,warm,59,NaN,436229.0,7.67,924051,36504,606,1.660092045,0.968937446,807067.0,2038.3,8.0,2009.0,"Saturday, August 29, 2009",10:54:00 PM,"Saturday, August 29, 2009",11:53:00 PM
1533,South Dakota,SD,MRO,islanding,West North Central,0.5,warm,181,NaN,436229.0,7.67,924051,36504,606,1.660092045,0.968937446,807067.0,2038.3,8.0,2009.0,"Saturday, August 29, 2009",11:00:00 AM,"Saturday, August 29, 2009",2:01:00 PM


In [215]:
 # combinbing date and time for outage start and restoration
outage_df['OUTAGE.START'] = pd.to_datetime(outage_df['OUTAGE.START.DATE'] + ' ' + outage_df['OUTAGE.START.TIME'],format="%A, %B %d, %Y %I:%M:%S %p")
outage_df['OUTAGE.RESTORATION'] =pd.to_datetime(outage_df['OUTAGE.RESTORATION.DATE'] + ' ' + outage_df['OUTAGE.RESTORATION.TIME'],format="%A, %B %d, %Y %I:%M:%S %p")

outage_df = outage_df.drop(columns =["OUTAGE.START.DATE",
"OUTAGE.START.TIME",
"OUTAGE.RESTORATION.DATE",
"OUTAGE.RESTORATION.TIME"],axis=1 )

In [216]:
for i,j in outage_df.dtypes.items():
    print(i,j)


U.S._STATE object
POSTAL.CODE object
NERC.REGION object
CAUSE.CATEGORY object
CLIMATE.REGION object
ANOMALY.LEVEL object
CLIMATE.CATEGORY object
OUTAGE.DURATION object
CUSTOMERS.AFFECTED float64
TOTAL.CUSTOMERS float64
TOTAL.PRICE object
TOTAL.SALES object
TOTAL.REALGSP object
UTIL.REALGSP object
UTIL.CONTRI object
PC.REALGSP.REL object
POPULATION float64
POPDEN_URBAN object
MONTH float64
YEAR float64
OUTAGE.START datetime64[ns]
OUTAGE.RESTORATION datetime64[ns]


In [217]:
# need to make numbers into floats
outage_df[['ANOMALY.LEVEL','OUTAGE.DURATION','TOTAL.PRICE','TOTAL.SALES','TOTAL.REALGSP', 'UTIL.REALGSP','POPDEN_URBAN','PC.REALGSP.REL','UTIL.CONTRI']] = (
    outage_df[['ANOMALY.LEVEL','OUTAGE.DURATION','TOTAL.PRICE','TOTAL.SALES','TOTAL.REALGSP','UTIL.REALGSP','POPDEN_URBAN','PC.REALGSP.REL','UTIL.CONTRI']].astype(float))


In [218]:
for i,j in outage_df.dtypes.items():
    print(i,j)


U.S._STATE object
POSTAL.CODE object
NERC.REGION object
CAUSE.CATEGORY object
CLIMATE.REGION object
ANOMALY.LEVEL float64
CLIMATE.CATEGORY object
OUTAGE.DURATION float64
CUSTOMERS.AFFECTED float64
TOTAL.CUSTOMERS float64
TOTAL.PRICE float64
TOTAL.SALES float64
TOTAL.REALGSP float64
UTIL.REALGSP float64
UTIL.CONTRI float64
PC.REALGSP.REL float64
POPULATION float64
POPDEN_URBAN float64
MONTH float64
YEAR float64
OUTAGE.START datetime64[ns]
OUTAGE.RESTORATION datetime64[ns]


In [219]:
print((outage_df['MONTH'] == 0).sum())
print((outage_df['YEAR'] == 0).sum())
print((outage_df["TOTAL.CUSTOMERS"] == 0).sum() )
print((outage_df['TOTAL.SALES'] == 0).sum())
print((outage_df['TOTAL.SALES'] == 0).sum())
print((outage_df['TOTAL.PRICE'] == 0).sum())
print((outage_df['PC.REALGSP.REL'] == 0).sum())
print((outage_df['UTIL.REALGSP'] == 0).sum())




0
0
0
0
0
0
0
0


In [220]:
(outage_df['OUTAGE.DURATION'] == 0).sum() # gave us 78
outage_df['OUTAGE.DURATION'] = outage_df['OUTAGE.DURATION'].replace({0:np.nan})


In [221]:
(outage_df["CUSTOMERS.AFFECTED"] == 0).sum() # gave us 212

outage_df["CUSTOMERS.AFFECTED"] = outage_df["CUSTOMERS.AFFECTED"].replace({0:np.nan})

In [222]:
for i in outage_df.columns:
    print(f"{i}: {outage_df[i].isna().sum()}")

U.S._STATE: 0
POSTAL.CODE: 0
NERC.REGION: 0
CAUSE.CATEGORY: 0
CLIMATE.REGION: 6
ANOMALY.LEVEL: 9
CLIMATE.CATEGORY: 9
OUTAGE.DURATION: 136
CUSTOMERS.AFFECTED: 655
TOTAL.CUSTOMERS: 0
TOTAL.PRICE: 22
TOTAL.SALES: 22
TOTAL.REALGSP: 0
UTIL.REALGSP: 0
UTIL.CONTRI: 0
PC.REALGSP.REL: 0
POPULATION: 0
POPDEN_URBAN: 0
MONTH: 9
YEAR: 0
OUTAGE.START: 9
OUTAGE.RESTORATION: 58


### Identifying Missingness Mechanisms For Customers Affected and Imputation

In [223]:
def tvd_perm_test(df, column,column_missing):
    np.random.seed(22)
    n_repetitions = 1000
    shuffled = df.copy()
    col_missing = column_missing
    col = column
    shuffled[f'{col_missing}_IS_MISSING'] = shuffled[col_missing].isna()
    
    tvds = []
    for _ in range(n_repetitions):
        
    
        shuffled[col] = np.random.permutation(shuffled[col])
        
      
        pivoted = (
            shuffled
            .pivot_table(index=col, columns=f'{col_missing}_IS_MISSING', aggfunc='size')
        )
        
        pivoted = pivoted / pivoted.sum()
        
        tvd = pivoted.diff(axis=1).iloc[:, -1].abs().sum() / 2
        tvds.append(tvd)
    return tvds
def pivot_table(df, col, col_missing):
    df_copy = df.copy()
    df_copy[f'{col_missing}_IS_MISSING'] = df_copy[col_missing].isna()
    pv = df_copy.pivot_table(index=col, columns=f'{col_missing}_IS_MISSING', aggfunc='size',fill_value=0)
    pv = pv/pv.sum()
    return pv

In [224]:
obs_pivot_y = pivot_table(outage_df, 'CLIMATE.CATEGORY', 'CUSTOMERS.AFFECTED')
obs_stat_y = obs_pivot_y.diff(axis=1).iloc[:, -1].abs().sum() / 2
obs_stat_y

0.02647273697964231

In [225]:
tvds_y = tvd_perm_test(outage_df,'CLIMATE.CATEGORY','CUSTOMERS.AFFECTED')

In [226]:
p_value = (tvds_y >= obs_stat_y).mean()
p_value

0.494

In [227]:
fig = px.histogram(pd.DataFrame({'tvds':tvds_y}),histnorm='probability', 
                   title='Empirical Distribution of the TVD')
fig.add_vline(x=obs_stat_y, line_color='red', line_width=2, opacity=1)
fig.add_annotation(text=f'<span style="color:red">Observed TVD = {round(obs_stat_y, 2)}</span>',
                   x=0.85 * obs_stat_y, showarrow=False, y=0.09)
fig.update_layout(yaxis_range=[0, 0.1])
fig.update_layout(xaxis_range=[0, 0.1])
fig.update_layout(showlegend=False)
fig.update_layout(
    xaxis_title="Generated TVDs"  
)

In [228]:
obs_pivot = pivot_table(outage_df, 'U.S._STATE', 'CUSTOMERS.AFFECTED')
observed_statistic = obs_pivot.diff(axis=1).iloc[:, -1].abs().sum() / 2
observed_statistic

0.38676497407706534

In [229]:
obs_pivot.plot(kind='barh', title='Gender by Missingness of Child Height (MAR Example)', barmode='group')

In [230]:
def tvd_perm_test(df, column,column_missing):
    np.random.seed(22)
    n_repetitions = 1000
    shuffled = df.copy()
    col_missing = column_missing
    col = column
    shuffled[f'{col_missing}_IS_MISSING'] = shuffled[col_missing].isna()
    
    tvds = []
    for _ in range(n_repetitions):
        
    
        shuffled[col] = np.random.permutation(shuffled[col])
        
      
        pivoted = (
            shuffled
            .pivot_table(index=col, columns=f'{col_missing}_IS_MISSING', aggfunc='size')
        )
        
        pivoted = pivoted / pivoted.sum()
        
        tvd = pivoted.diff(axis=1).iloc[:, -1].abs().sum() / 2
        tvds.append(tvd)
    return tvds

In [231]:
tvds = tvd_perm_test(outage_df, 'U.S._STATE','CUSTOMERS.AFFECTED')

In [232]:
p_value = (tvds >= observed_statistic).mean()
p_value

0.0

In [233]:

fig = px.histogram(pd.DataFrame({'tvds':tvds}),histnorm='probability', 
                   title='Empirical Distribution of the TVD')
fig.add_vline(x=observed_statistic, line_color='red', line_width=2, opacity=1)
fig.add_annotation(text=f'<span style="color:red">Observed TVD = {round(observed_statistic, 2)}</span>',
                   x=0.95 * observed_statistic, showarrow=False, y=0.12)
fig.update_layout(yaxis_range=[0, 0.13])
fig.update_layout(xaxis_range=[0, 0.4])
fig.update_layout(showlegend=False)
fig.update_layout(
    xaxis_title="Generated TVDs"  
)

In [234]:
pv_cause = pivot_table(outage_df, 'CAUSE.CATEGORY', 'CUSTOMERS.AFFECTED')
obs_stat_cause = pv_cause.diff(axis=1).iloc[:, -1].abs().sum() / 2
obs_stat_cause

0.7557790341210083

In [235]:
pv_cause.plot(kind='barh', title='Gender by Missingness of Child Height (MAR Example)', barmode='group')

In [236]:
tvds2 = tvd_perm_test(outage_df, 'CAUSE.CATEGORY','CUSTOMERS.AFFECTED')

In [237]:
p_value2 =(tvds >= obs_stat_cause).mean()
p_value2

0.0

In [238]:
fig = px.histogram(pd.DataFrame({'tvds':tvds2}),histnorm='probability',nbins = 20,
                   title='Empirical Distribution of the TVD')
fig.add_vline(x=obs_stat_cause, line_color='red', line_width=2, opacity=1)
fig.add_annotation(text=f'<span style="color:red">Observed TVD = {round(obs_stat_cause, 2)}</span>',
                   x=0.95 * obs_stat_cause, showarrow=False, y=0.14)
fig.update_layout(yaxis_range=[0, 0.15])
fig.update_layout(xaxis_range=[0, 0.8])
fig.update_layout(showlegend=False)
fig.update_layout(
    xaxis_title="Generated TVDs"  
)

In [239]:
def prob_impute(s):
    s = s.copy()
    
    num_null = s.isna().sum()
    
    fill_values = np.random.choice(s.dropna(), num_null)
    
    s[s.isna()] = fill_values
    return s
def one_impute(df, column, seed,groupby_col=None):
    np.random.seed(seed)
    df_copy = df.copy()

    if groupby_col:
        df_copy[column] = df_copy.groupby(groupby_col)[column].transform(lambda s: prob_impute(s))
    else:
        df_copy[column] =  prob_impute(df_copy[column])
    
    return df_copy[column]

In [240]:
imputed_cus_aff = one_impute(outage_df, 'CUSTOMERS.AFFECTED',23,'CAUSE.CATEGORY')
imputed_cus_df = pd.DataFrame({"CUSTOMERS.AFFECTED":imputed_cus_aff})
imputed_cus_df

,CUSTOMERS.AFFECTED
1,70000.0
2,215.0
3,70000.0
4,68200.0
5,250000.0
...,...
1530,34500.0
1531,1.0
1532,4.0
1533,15000.0


In [241]:
def multiple_imputation(df, column, seed,mult,num_imputations=5, groupby_col=None):
    imputed_dfs = [one_impute(df, column,seed + mult*i, groupby_col) for i in range(num_imputations)]
    return np.mean(imputed_dfs, axis=0) 
mimputed_cus_aff = multiple_imputation(outage_df, 'CUSTOMERS.AFFECTED',23,2,50,'CAUSE.CATEGORY')
mimputed_cus_df = pd.DataFrame({"CUSTOMERS.AFFECTED":mimputed_cus_aff })
mimputed_cus_df

,CUSTOMERS.AFFECTED
0,70000.00
1,13896.36
2,70000.00
3,68200.00
4,250000.00
...,...
1529,34500.00
1530,1.00
1531,5597.56
1532,8297.94


In [242]:
def multiple_kdes(df_map, col, title=""):
    values = [df_map[key][col].dropna() for key in df_map]
    labels = list(df_map.keys())
    fig = ff.create_distplot(
        hist_data=values,
        group_labels=labels,
        show_rug=False,
        show_hist=False,
        colors=px.colors.qualitative.Dark2[: len(df_map)],
    )
 
    return fig.update_layout(title=title).update_xaxes(title=col)



multiple_kdes({'Original': outage_df, 'MAR, Filled, single_prop': imputed_cus_df, 'MAR, Filled, mult_prop':mimputed_cus_df},'CUSTOMERS.AFFECTED')

In [243]:
outage_df['CUSTOMERS.AFFECTED.FILLED'] = mimputed_cus_aff

### Identifying Missingness Mechanisms For Outage Duration and Imputation

In [244]:
obs_pivot_c = pivot_table(outage_df, 'CAUSE.CATEGORY', 'OUTAGE.DURATION')
obs_stat_c = obs_pivot_c.diff(axis=1).iloc[:, -1].abs().sum() / 2

dur_tvds_c = tvd_perm_test(outage_df, 'CAUSE.CATEGORY','OUTAGE.DURATION')


(dur_tvds_c >= obs_stat_c).mean()

0.0

In [245]:
dur_tvds_s = tvd_perm_test(outage_df, 'U.S._STATE','OUTAGE.DURATION')

obs_pivot_s = pivot_table(outage_df, 'U.S._STATE', 'OUTAGE.DURATION')
obs_stat_s = obs_pivot_s.diff(axis=1).iloc[:, -1].abs().sum() / 2
obs_stat_s

(dur_tvds_s >= obs_stat_s).mean()

0.0

In [246]:
imputed_out = one_impute(outage_df, 'OUTAGE.DURATION',30,'CAUSE.CATEGORY')
imputed_out_df = pd.DataFrame({'OUTAGE.DURATION':imputed_out })
imputed_out_df

,OUTAGE.DURATION
1,3060.0
2,1.0
3,3000.0
4,2550.0
5,1740.0
...,...
1530,720.0
1531,660.0
1532,59.0
1533,181.0


In [247]:
mimputed_out = multiple_imputation(outage_df, 'OUTAGE.DURATION',30,3,50,'CAUSE.CATEGORY')
mimputed_out_df = pd.DataFrame({'OUTAGE.DURATION':mimputed_out })
mimputed_out_df

,OUTAGE.DURATION
0,3060.00
1,1.00
2,3000.00
3,2550.00
4,1740.00
...,...
1529,720.00
1530,14907.46
1531,59.00
1532,181.00


In [248]:
def multiple_kdes(df_map, col, title=""):
    values = [df_map[key][col].dropna() for key in df_map]
    labels = list(df_map.keys())
    fig = ff.create_distplot(
        hist_data=values,
        group_labels=labels,
        show_rug=False,
        show_hist=False,
        colors=px.colors.qualitative.Dark2[: len(df_map)],
    )
    min_value = min(df[col].min() for df in df_map.values())
    max_value = max(df[col].max() for df in df_map.values())
    return fig.update_layout(title=title).update_xaxes(title=col, range=[min_value, max_value])

multiple_kdes({'Original': outage_df, 'MAR, Filled, Single Imputation': imputed_out_df, 'MAR, Filled, Multiple Imputation':mimputed_out_df},'OUTAGE.DURATION')

In [249]:
outage_df["OUTAGE.DURATION.FILLED"] = mimputed_out

In [250]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

impact_features = ['CUSTOMERS.AFFECTED.FILLED', 'OUTAGE.DURATION.FILLED']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(outage_df[impact_features])
X_scaled

array([[-0.21437911,  0.05384061],
       [-0.44098305, -0.46226863],
       [-0.21437911,  0.04371751],
       ...,
       [-0.4745021 , -0.45248297],
       [-0.4635952 , -0.43189934],
       [-0.43946179, -0.12392098]])

In [251]:
pca = PCA(n_components=1)
impact_score = pca.fit_transform(X_scaled)

outage_df['IMPACT_SCORE'] = impact_score
outage_df['IMPACT_SCORE']

1      -0.113518
2      -0.638695
3      -0.120676
4      -0.179503
5       0.243088
          ...   
1530   -0.494073
1531    1.099987
1532   -0.655477
1533   -0.633210
1534   -0.398372
Name: IMPACT_SCORE, Length: 1534, dtype: float64

In [252]:
outage_df=outage_df.drop(['OUTAGE.DURATION.FILLED','CUSTOMERS.AFFECTED.FILLED','OUTAGE.DURATION','CUSTOMERS.AFFECTED'],axis=1)

In [253]:
outage_df

,U.S._STATE,POSTAL.CODE,NERC.REGION,CAUSE.CATEGORY,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,TOTAL.CUSTOMERS,TOTAL.PRICE,TOTAL.SALES,TOTAL.REALGSP,UTIL.REALGSP,UTIL.CONTRI,PC.REALGSP.REL,POPULATION,POPDEN_URBAN,MONTH,YEAR,OUTAGE.START,OUTAGE.RESTORATION,IMPACT_SCORE
1,Minnesota,MN,MRO,severe weather,East North Central,-0.3,normal,2595696.0,9.28,6562520.0,274182.0,4802.0,1.751391,1.077376,5348119.0,2279.0,7.0,2011.0,2011-07-01 17:00:00,2011-07-03 20:00:00,-0.113518
2,Minnesota,MN,MRO,intentional attack,East North Central,-0.1,normal,2640737.0,9.28,5284231.0,291955.0,5226.0,1.790002,1.089792,5457125.0,2279.0,5.0,2014.0,2014-05-11 18:38:00,2014-05-11 18:39:00,-0.638695
3,Minnesota,MN,MRO,severe weather,East North Central,-1.5,cold,2586905.0,8.15,5222116.0,267895.0,4571.0,1.706266,1.066826,5310903.0,2279.0,10.0,2010.0,2010-10-26 20:00:00,2010-10-28 22:00:00,-0.120676
4,Minnesota,MN,MRO,severe weather,East North Central,-0.1,normal,2606813.0,9.19,5787064.0,277627.0,5364.0,1.932089,1.071476,5380443.0,2279.0,6.0,2012.0,2012-06-19 04:30:00,2012-06-20 23:00:00,-0.179503
5,Minnesota,MN,MRO,severe weather,East North Central,1.2,warm,2673531.0,10.43,5970339.0,292023.0,4873.0,1.668704,1.092027,5489594.0,2279.0,7.0,2015.0,2015-07-18 02:00:00,2015-07-19 07:00:00,0.243088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1530,North Dakota,ND,MRO,public appeal,West North Central,-0.9,cold,394394.0,7.56,1313678.0,39067.0,934.0,2.390765,1.198083,685326.0,2192.2,12.0,2011.0,2011-12-06 08:00:00,2011-12-06 20:00:00,-0.494073
1531,North Dakota,ND,MRO,fuel supply emergency,West North Central,NaN,NaN,366037.0,NaN,NaN,27868.0,1019.0,3.656524,0.877405,649422.0,2192.2,NaN,2006.0,NaT,NaT,1.099987
1532,South Dakota,SD,RFC,islanding,West North Central,0.5,warm,436229.0,7.67,924051.0,36504.0,606.0,1.660092,0.968937,807067.0,2038.3,8.0,2009.0,2009-08-29 22:54:00,2009-08-29 23:53:00,-0.655477
1533,South Dakota,SD,MRO,islanding,West North Central,0.5,warm,436229.0,7.67,924051.0,36504.0,606.0,1.660092,0.968937,807067.0,2038.3,8.0,2009.0,2009-08-29 11:00:00,2009-08-29 14:01:00,-0.633210


In [254]:
print(outage_df.head().to_markdown(index=False))

| U.S._STATE   | POSTAL.CODE   | NERC.REGION   | CAUSE.CATEGORY     | CLIMATE.REGION     |   ANOMALY.LEVEL | CLIMATE.CATEGORY   |   TOTAL.CUSTOMERS |   TOTAL.PRICE |   TOTAL.SALES |   TOTAL.REALGSP |   UTIL.REALGSP |   UTIL.CONTRI |   PC.REALGSP.REL |   POPULATION |   POPDEN_URBAN |   MONTH |   YEAR | OUTAGE.START        | OUTAGE.RESTORATION   |   IMPACT_SCORE |
|:-------------|:--------------|:--------------|:-------------------|:-------------------|----------------:|:-------------------|------------------:|--------------:|--------------:|----------------:|---------------:|--------------:|-----------------:|-------------:|---------------:|--------:|-------:|:--------------------|:---------------------|---------------:|
| Minnesota    | MN            | MRO           | severe weather     | East North Central |            -0.3 | normal             |       2.5957e+06  |          9.28 |   6.56252e+06 |          274182 |           4802 |       1.75139 |          1.07738 |  5.34812e+06 |    

### Univariate Analysis

In [255]:
fig = px.histogram(outage_df,x='IMPACT_SCORE',histnorm='probability',marginal='box',
                   title='Distribution of Impact Scores')
fig.update_layout(
    width=800,  
    height=600    
)
fig.show()

In [256]:
outage_df['IMPACT_SCORE'].mean()

-4.631960467797263e-18

In [257]:
cause_counts = (outage_df.groupby('CAUSE.CATEGORY').size().reset_index()
                .rename(columns = {'CAUSE.CATEGORY': "Cause Category", 0:"Number of Outages"})
                .sort_values("Number of Outages",ascending=False))

dist_causes = px.bar(cause_counts, x='Cause Category', y="Number of Outages", title="Number of Power Outages for Each Major Cause")
dist_causes.update_layout(
    width=800,  
    height=600    
)
dist_causes.show()


### Bivariate Analysis

In [258]:
fig = px.box(outage_df, x="CAUSE.CATEGORY", y="IMPACT_SCORE",
             title="Box Plots of Impact Scores by Cause Category",
             labels={"CAUSE.CATEGORY": "Cause Category", "IMPACT_SCORE": "Impact Score"},
             color="CAUSE.CATEGORY")  # Colors by category
fig.update_layout(
    width=800,  
    height=600   
)
fig.show()


In [259]:


fig = px.box(outage_df, x="CLIMATE.REGION", y="IMPACT_SCORE",
             title="Box Plots of Impact Scores by Climate Region",
             labels={"CLIMATE.REGION": "Climate Region", "IMPACT_SCORE": "Impact Score"},
             color="CLIMATE.REGION")  # Colors by category

fig.show()


In [260]:

fig = px.box(outage_df, x="NERC.REGION", y="IMPACT_SCORE",
             title="Box Plots of Impact Scores by NERC Region",
             labels={"NERC.REGION": "NERC Region", "IMPACT_SCORE": "Impact Score"},
             color="NERC.REGION")  # Colors by category

fig.show()

In [261]:

fig = px.box(outage_df, x="CLIMATE.CATEGORY", y="IMPACT_SCORE",
             title="Box Plots of Impact Scores by Climate Category",
             labels={"CLIMATE.CATEGORY": "Climate Category", "IMPACT_SCORE": "Impact Score"},
             color="CLIMATE.CATEGORY")  # Colors by category

fig.show()

In [262]:

fig = px.scatter(outage_df, x="TOTAL.SALES", y="IMPACT_SCORE",
                 title="Impact Score vs. Total Sales",
                 labels={"TOTAL.SALES": "Total Sales", "IMPACT_SCORE": "Impact Score"})  # Adds an optional trendline

fig.show()



In [263]:

fig = px.scatter(outage_df, x="POPDEN_URBAN", y="IMPACT_SCORE",
                 title="Impact Score vs. Urban Population Density",
                 labels={"POPDEN_URBAN": "Urban Population Density", "IMPACT_SCORE": "Impact Score"})  # Adds an optional trendline

fig.show()



In [264]:

impact_avg = outage_df.groupby("YEAR")["IMPACT_SCORE"].mean().reset_index()

fig = px.line(impact_avg, x="YEAR", y="IMPACT_SCORE",
              title="Trend of Impact Score Over the Years",
              labels={"YEAR": "Year", "IMPACT_SCORE": "Average Impact Score"},
              markers=True)
fig.update_layout(
    width=800,  
    height=600   
)

#fig.show()


In [265]:

impact_avg = outage_df.groupby("POSTAL.CODE").agg(
    IMPACT_SCORE=('IMPACT_SCORE', 'mean'),
    State_Name=('U.S._STATE', 'first')  
).reset_index()

fig = px.choropleth(impact_avg,
                     locations="POSTAL.CODE", 
                     locationmode="USA-states",
                     color="IMPACT_SCORE",  
                     color_continuous_scale="Reds",  
                     title="Average Impact Score by U.S. State",
                     labels={"IMPACT_SCORE": "Average Impact Score"},
                     scope="usa",
                     hover_name="State_Name"  )  
fig.update_layout(
    width=800,  
    height=600   
)


fig.show()
impact_avg

,POSTAL.CODE,IMPACT_SCORE,State_Name
0,AK,-0.398372,Alaska
1,AL,-0.262630,Alabama
2,AR,-0.318978,Arkansas
3,AZ,-0.040913,Arizona
4,CA,0.046270,California
5,CO,-0.208507,Colorado
6,CT,-0.338605,Connecticut
7,DC,0.395542,District of Columbia
8,DE,-0.548107,Delaware
9,FL,0.690109,Florida


In [266]:
impact_avg.sort_values('IMPACT_SCORE')

,POSTAL.CODE,IMPACT_SCORE,State_Name
26,MT,-0.644611,Montana
40,SD,-0.644344,South Dakota
45,VT,-0.625497,Vermont
25,MS,-0.622374,Mississippi
49,WY,-0.575741,Wyoming
43,UT,-0.568645,Utah
30,NH,-0.557173,New Hampshire
8,DE,-0.548107,Delaware
33,NV,-0.528472,Nevada
13,ID,-0.494814,Idaho


### Interesting Aggregates

In [267]:
agg_im= outage_df.pivot_table(index='NERC.REGION',columns='CAUSE.CATEGORY',values="IMPACT_SCORE",aggfunc='mean')
agg_im

CAUSE.CATEGORY,equipment failure,fuel supply emergency,intentional attack,islanding,public appeal,severe weather,system operability disruption
NERC.REGION,,,,,,,
ASCC,-0.398372,NaN,NaN,NaN,NaN,NaN,NaN
ECAR,-0.230388,NaN,-0.418185,NaN,NaN,0.518388,2.915342
FRCC,-0.160836,NaN,-0.625005,NaN,-0.116905,1.447440,-0.496310
"FRCC, SERC",NaN,NaN,NaN,NaN,NaN,NaN,0.050804
HECO,NaN,NaN,NaN,NaN,NaN,-0.031349,-0.566547
HI,NaN,NaN,NaN,NaN,NaN,0.324253,NaN
MRO,NaN,0.775508,-0.376839,-0.647669,-0.552291,0.049756,NaN
NPCC,-0.305414,1.033702,-0.592205,-0.545928,-0.310625,0.207407,0.783275
PR,NaN,NaN,NaN,NaN,NaN,-0.480671,NaN


In [268]:
print(agg_im.to_markdown(index=True))

| NERC.REGION   |   equipment failure |   fuel supply emergency |   intentional attack |   islanding |   public appeal |   severe weather |   system operability disruption |
|:--------------|--------------------:|------------------------:|---------------------:|------------:|----------------:|-----------------:|--------------------------------:|
| ASCC          |           -0.398372 |              nan        |           nan        |  nan        |      nan        |      nan         |                     nan         |
| ECAR          |           -0.230388 |              nan        |            -0.418185 |  nan        |      nan        |        0.518388  |                       2.91534   |
| FRCC          |           -0.160836 |              nan        |            -0.625005 |  nan        |       -0.116905 |        1.44744   |                      -0.49631   |
| FRCC, SERC    |          nan        |              nan        |           nan        |  nan        |      nan        |      nan 

## Step 3: Assessment of Missingness

In [269]:
def tvd_perm_test(df, column,column_missing):
    np.random.seed(22)
    n_repetitions = 1000
    shuffled = df.copy()
    col_missing = column_missing
    col = column
    shuffled[f'{col_missing}_IS_MISSING'] = shuffled[col_missing].isna()
    
    tvds = []
    for _ in range(n_repetitions):
        
    
        shuffled[col] = np.random.permutation(shuffled[col])
        
      
        pivoted = (
            shuffled
            .pivot_table(index=col, columns=f'{col_missing}_IS_MISSING', aggfunc='size')
        )
        
        pivoted = pivoted / pivoted.sum()
        
        tvd = pivoted.diff(axis=1).iloc[:, -1].abs().sum() / 2
        tvds.append(tvd)
    return tvds
def pivot_table(df, col, col_missing):
    df_copy = df.copy()
    df_copy[f'{col_missing}_IS_MISSING'] = df_copy[col_missing].isna()
    pv = df_copy.pivot_table(index=col, columns=f'{col_missing}_IS_MISSING', aggfunc='size',fill_value=0)
    pv = pv/pv.sum()
    return pv

In [270]:
for i in outage_df.columns:
    print(f"{i}: {outage_df[i].isna().sum()}")

U.S._STATE: 0
POSTAL.CODE: 0
NERC.REGION: 0
CAUSE.CATEGORY: 0
CLIMATE.REGION: 6
ANOMALY.LEVEL: 9
CLIMATE.CATEGORY: 9
TOTAL.CUSTOMERS: 0
TOTAL.PRICE: 22
TOTAL.SALES: 22
TOTAL.REALGSP: 0
UTIL.REALGSP: 0
UTIL.CONTRI: 0
PC.REALGSP.REL: 0
POPULATION: 0
POPDEN_URBAN: 0
MONTH: 9
YEAR: 0
OUTAGE.START: 9
OUTAGE.RESTORATION: 58
IMPACT_SCORE: 0


In [271]:
outage_df[outage_df['CLIMATE.REGION'].isna()]

,U.S._STATE,POSTAL.CODE,NERC.REGION,CAUSE.CATEGORY,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,TOTAL.CUSTOMERS,TOTAL.PRICE,TOTAL.SALES,TOTAL.REALGSP,UTIL.REALGSP,UTIL.CONTRI,PC.REALGSP.REL,POPULATION,POPDEN_URBAN,MONTH,YEAR,OUTAGE.START,OUTAGE.RESTORATION,IMPACT_SCORE
1516,Hawaii,HI,HI,severe weather,NaN,-0.7,cold,472025.0,25.78,835405.0,67364.0,1646.0,2.443442,1.044710,1332213.0,3180.8,12.0,2008.0,2008-12-26 18:13:00,2008-12-27 17:00:00,0.324253
1517,Hawaii,HI,PR,severe weather,NaN,-0.4,normal,478272.0,31.29,852355.0,67686.0,1935.0,2.858789,1.032026,1378227.0,3180.8,5.0,2011.0,2011-05-02 17:06:00,2011-05-02 20:00:00,-0.480671
1518,Hawaii,HI,HECO,severe weather,NaN,0.7,warm,463615.0,20.54,922915.0,65956.0,1436.0,2.177209,1.029626,1309731.0,3180.8,10.0,2006.0,2006-10-15 07:09:00,2006-10-15 16:12:00,-0.442686
1519,Hawaii,HI,HECO,system operability disruption,NaN,0.0,normal,463615.0,21.45,891580.0,65956.0,1436.0,2.177209,1.029626,1309731.0,3180.8,6.0,2006.0,2006-06-01 14:12:00,2006-06-01 18:09:00,-0.566547
1520,Hawaii,HI,HECO,severe weather,NaN,0.7,warm,463615.0,20.54,922915.0,65956.0,1436.0,2.177209,1.029626,1309731.0,3180.8,10.0,2006.0,2006-10-15 07:09:00,2006-10-16 14:55:00,0.379988
1534,Alaska,AK,ASCC,equipment failure,NaN,NaN,NaN,273530.0,NaN,NaN,36046.0,724.0,2.008545,1.282847,627963.0,1802.6,NaN,2000.0,NaT,NaT,-0.398372


In [272]:
outage_df[outage_df['U.S._STATE'] == 'Alaska']

,U.S._STATE,POSTAL.CODE,NERC.REGION,CAUSE.CATEGORY,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,TOTAL.CUSTOMERS,TOTAL.PRICE,TOTAL.SALES,TOTAL.REALGSP,UTIL.REALGSP,UTIL.CONTRI,PC.REALGSP.REL,POPULATION,POPDEN_URBAN,MONTH,YEAR,OUTAGE.START,OUTAGE.RESTORATION,IMPACT_SCORE
1534,Alaska,AK,ASCC,equipment failure,NaN,NaN,NaN,273530.0,NaN,NaN,36046.0,724.0,2.008545,1.282847,627963.0,1802.6,NaN,2000.0,NaT,NaT,-0.398372


In [273]:
outage_df[outage_df['TOTAL.SALES'].isna()].iloc[10:20]

,U.S._STATE,POSTAL.CODE,NERC.REGION,CAUSE.CATEGORY,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,TOTAL.CUSTOMERS,TOTAL.PRICE,TOTAL.SALES,TOTAL.REALGSP,UTIL.REALGSP,UTIL.CONTRI,PC.REALGSP.REL,POPULATION,POPDEN_URBAN,MONTH,YEAR,OUTAGE.START,OUTAGE.RESTORATION,IMPACT_SCORE
763,North Carolina,NC,SERC,intentional attack,Southeast,-0.3,normal,5106195.0,NaN,NaN,448866.0,7443.0,1.658179,0.872365,10156689.0,1367.2,7.0,2016.0,2016-07-07 05:53:00,2016-07-07 08:40:00,-0.619052
767,North Carolina,NC,SERC,severe weather,Southeast,NaN,NaN,4105711.0,NaN,NaN,341051.0,8262.0,2.422512,0.943144,8081614.0,1367.2,NaN,2000.0,NaT,NaT,-0.204062
828,Oregon,OR,WECC,intentional attack,Northwest,-0.3,normal,1962944.0,NaN,NaN,207367.0,3179.0,1.533031,1.001796,4085989.0,2804.9,7.0,2016.0,2016-07-02 04:00:00,2016-07-04 00:40:00,-0.303485
888,Delaware,DE,RFC,system operability disruption,Northeast,NaN,NaN,377219.0,NaN,NaN,50338.0,1016.0,2.018356,1.430618,786373.0,1838.3,NaN,2000.0,NaT,NaT,0.706756
901,Delaware,DE,RFC,intentional attack,Northeast,-0.3,normal,473473.0,NaN,NaN,60570.0,947.0,1.563480,1.254994,952698.0,1838.3,7.0,2016.0,2016-07-21 06:18:00,2016-07-21 14:45:00,-0.548645
1319,Virginia,VA,SERC,equipment failure,Southeast,NaN,NaN,3122909.0,NaN,NaN,337211.0,6978.0,2.069328,1.060588,7105817.0,2265.2,NaN,2000.0,NaT,NaT,0.040699
1420,Oklahoma,OK,SPP,severe weather,South,-0.3,normal,2033942.0,NaN,NaN,174173.0,4334.0,2.488331,0.876786,3921207.0,1901.7,7.0,2016.0,2016-07-14 14:44:00,2016-07-15 04:00:00,-0.306602
1507,Kansas,KS,SPP,severe weather,South,NaN,NaN,1379347.0,NaN,NaN,109966.0,2462.0,2.238874,0.898619,2713535.0,2176.5,NaN,2002.0,NaT,NaT,0.502689
1521,Idaho,ID,WECC,system operability disruption,Northwest,-0.3,normal,849763.0,NaN,NaN,60911.0,960.0,1.576070,0.715673,1680026.0,2216.8,7.0,2016.0,2016-07-19 15:45:00,2016-07-19 19:29:00,-0.382543
1529,Idaho,ID,WECC,system operability disruption,Northwest,-0.3,normal,849763.0,NaN,NaN,60911.0,960.0,1.576070,0.715673,1680026.0,2216.8,7.0,2016.0,2016-07-19 15:45:00,2016-07-19 19:25:00,-0.264173


In [274]:
outage_df[outage_df['OUTAGE.START'].isna()]

,U.S._STATE,POSTAL.CODE,NERC.REGION,CAUSE.CATEGORY,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,TOTAL.CUSTOMERS,TOTAL.PRICE,TOTAL.SALES,TOTAL.REALGSP,UTIL.REALGSP,UTIL.CONTRI,PC.REALGSP.REL,POPULATION,POPDEN_URBAN,MONTH,YEAR,OUTAGE.START,OUTAGE.RESTORATION,IMPACT_SCORE
240,Texas,TX,FRCC,equipment failure,South,NaN,NaN,9299829.0,NaN,NaN,944631.0,30908.0,3.271965,1.007979,20944499.0,2435.3,NaN,2000.0,NaT,NaT,-0.325723
340,Alabama,AL,SERC,severe weather,Southeast,NaN,NaN,2262753.0,NaN,NaN,150090.0,5704.0,3.800386,0.753425,4452173.0,1278.5,NaN,2000.0,NaT,NaT,0.185409
366,Illinois,IL,SERC,severe weather,Central,NaN,NaN,5282401.0,NaN,NaN,612709.0,15815.0,2.581160,1.101263,12434161.0,2877.6,NaN,2000.0,NaT,NaT,-0.217359
767,North Carolina,NC,SERC,severe weather,Southeast,NaN,NaN,4105711.0,NaN,NaN,341051.0,8262.0,2.422512,0.943144,8081614.0,1367.2,NaN,2000.0,NaT,NaT,-0.204062
888,Delaware,DE,RFC,system operability disruption,Northeast,NaN,NaN,377219.0,NaN,NaN,50338.0,1016.0,2.018356,1.430618,786373.0,1838.3,NaN,2000.0,NaT,NaT,0.706756
1319,Virginia,VA,SERC,equipment failure,Southeast,NaN,NaN,3122909.0,NaN,NaN,337211.0,6978.0,2.069328,1.060588,7105817.0,2265.2,NaN,2000.0,NaT,NaT,0.040699
1507,Kansas,KS,SPP,severe weather,South,NaN,NaN,1379347.0,NaN,NaN,109966.0,2462.0,2.238874,0.898619,2713535.0,2176.5,NaN,2002.0,NaT,NaT,0.502689
1531,North Dakota,ND,MRO,fuel supply emergency,West North Central,NaN,NaN,366037.0,NaN,NaN,27868.0,1019.0,3.656524,0.877405,649422.0,2192.2,NaN,2006.0,NaT,NaT,1.099987
1534,Alaska,AK,ASCC,equipment failure,NaN,NaN,NaN,273530.0,NaN,NaN,36046.0,724.0,2.008545,1.282847,627963.0,1802.6,NaN,2000.0,NaT,NaT,-0.398372


In [275]:
outage_df[outage_df['OUTAGE.RESTORATION'].isna()]

,U.S._STATE,POSTAL.CODE,NERC.REGION,CAUSE.CATEGORY,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,TOTAL.CUSTOMERS,TOTAL.PRICE,TOTAL.SALES,TOTAL.REALGSP,UTIL.REALGSP,UTIL.CONTRI,PC.REALGSP.REL,POPULATION,POPDEN_URBAN,MONTH,YEAR,OUTAGE.START,OUTAGE.RESTORATION,IMPACT_SCORE
23,Tennessee,TN,SERC,intentional attack,Central,1.2,warm,3260708.0,9.70,9499547.0,282752.0,1692.0,0.598404,0.851798,6600299.0,1450.3,7.0,2015.0,2015-07-30 13:00:00,NaT,-0.571327
37,Tennessee,TN,SERC,system operability disruption,Central,-0.3,normal,3293512.0,NaN,NaN,290712.0,1813.0,0.623641,0.863008,6649404.0,1450.3,7.0,2016.0,2016-07-13 15:00:00,NaT,-0.004068
48,Tennessee,TN,SERC,intentional attack,Central,1.1,warm,3293512.0,8.80,6776403.0,290712.0,1813.0,0.623641,0.863008,6649404.0,1450.3,4.0,2016.0,2016-04-27 13:36:00,NaT,-0.575792
50,Wisconsin,WI,MRO,fuel supply emergency,East North Central,0.0,normal,2982802.0,10.98,5757447.0,268742.0,4680.0,1.741447,0.950806,5759432.0,2123.3,6.0,2014.0,2014-06-27 13:21:00,NaT,1.564227
183,Texas,TX,WECC,fuel supply emergency,South,-0.9,cold,10748834.0,10.30,33193019.0,1169399.0,27695.0,2.368311,0.998819,23831983.0,2435.3,9.0,2007.0,2007-09-06 20:00:00,NaT,1.256561
193,Texas,TX,TRE,fuel supply emergency,South,-0.2,normal,11672996.0,8.55,27846934.0,1421759.0,28121.0,1.977902,1.074372,26979078.0,2435.3,4.0,2014.0,2014-04-03 00:00:00,NaT,0.752757
233,Texas,TX,TRE,fuel supply emergency,South,0.0,normal,11672996.0,9.20,34707247.0,1421759.0,28121.0,1.977902,1.074372,26979078.0,2435.3,6.0,2014.0,2014-06-06 13:00:00,NaT,0.321147
240,Texas,TX,FRCC,equipment failure,South,NaN,NaN,9299829.0,NaN,NaN,944631.0,30908.0,3.271965,1.007979,20944499.0,2435.3,NaN,2000.0,NaT,NaT,-0.325723
283,Texas,TX,TRE,severe weather,South,2.3,warm,11849725.0,8.20,28745852.0,1488049.0,31256.0,2.100468,1.077502,27469114.0,2435.3,12.0,2015.0,2015-12-26 19:30:00,NaT,-0.031025
302,Indiana,IN,ECAR,equipment failure,Central,-0.5,cold,2865343.0,5.39,9080103.0,251606.0,6964.0,2.767820,0.923053,6091866.0,1860.0,8.0,2000.0,2000-08-28 23:00:00,NaT,-0.263195


In [276]:
outage_df[outage_df['MONTH'].isna()]

,U.S._STATE,POSTAL.CODE,NERC.REGION,CAUSE.CATEGORY,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,TOTAL.CUSTOMERS,TOTAL.PRICE,TOTAL.SALES,TOTAL.REALGSP,UTIL.REALGSP,UTIL.CONTRI,PC.REALGSP.REL,POPULATION,POPDEN_URBAN,MONTH,YEAR,OUTAGE.START,OUTAGE.RESTORATION,IMPACT_SCORE
240,Texas,TX,FRCC,equipment failure,South,NaN,NaN,9299829.0,NaN,NaN,944631.0,30908.0,3.271965,1.007979,20944499.0,2435.3,NaN,2000.0,NaT,NaT,-0.325723
340,Alabama,AL,SERC,severe weather,Southeast,NaN,NaN,2262753.0,NaN,NaN,150090.0,5704.0,3.800386,0.753425,4452173.0,1278.5,NaN,2000.0,NaT,NaT,0.185409
366,Illinois,IL,SERC,severe weather,Central,NaN,NaN,5282401.0,NaN,NaN,612709.0,15815.0,2.581160,1.101263,12434161.0,2877.6,NaN,2000.0,NaT,NaT,-0.217359
767,North Carolina,NC,SERC,severe weather,Southeast,NaN,NaN,4105711.0,NaN,NaN,341051.0,8262.0,2.422512,0.943144,8081614.0,1367.2,NaN,2000.0,NaT,NaT,-0.204062
888,Delaware,DE,RFC,system operability disruption,Northeast,NaN,NaN,377219.0,NaN,NaN,50338.0,1016.0,2.018356,1.430618,786373.0,1838.3,NaN,2000.0,NaT,NaT,0.706756
1319,Virginia,VA,SERC,equipment failure,Southeast,NaN,NaN,3122909.0,NaN,NaN,337211.0,6978.0,2.069328,1.060588,7105817.0,2265.2,NaN,2000.0,NaT,NaT,0.040699
1507,Kansas,KS,SPP,severe weather,South,NaN,NaN,1379347.0,NaN,NaN,109966.0,2462.0,2.238874,0.898619,2713535.0,2176.5,NaN,2002.0,NaT,NaT,0.502689
1531,North Dakota,ND,MRO,fuel supply emergency,West North Central,NaN,NaN,366037.0,NaN,NaN,27868.0,1019.0,3.656524,0.877405,649422.0,2192.2,NaN,2006.0,NaT,NaT,1.099987
1534,Alaska,AK,ASCC,equipment failure,NaN,NaN,NaN,273530.0,NaN,NaN,36046.0,724.0,2.008545,1.282847,627963.0,1802.6,NaN,2000.0,NaT,NaT,-0.398372


In [277]:
obs_pivot_o = pivot_table(outage_df, 'CAUSE.CATEGORY', 'OUTAGE.RESTORATION')

obs_stat_o = obs_pivot_o.diff(axis=1).iloc[:, -1].abs().sum() / 2
obs_stat_o

0.2520091580226147

In [278]:
cause_dist = obs_pivot_o.plot(kind='barh', title='Cause Category by Missingness of Outage Restoration', barmode='group')

cause_dist.update_layout(
    width=800,  
    height=600   
)

In [279]:
tvds_o = tvd_perm_test(outage_df, 'CAUSE.CATEGORY', 'OUTAGE.RESTORATION') 

In [280]:
(tvds_o >= obs_stat_o).mean()

0.002

In [281]:
fig = px.histogram(pd.DataFrame({'tvds':tvds_o}),histnorm='probability', 
                   title='Empirical Distribution of the TVD')
fig.add_vline(x=obs_stat_o, line_color='red', line_width=2, opacity=1)
fig.add_annotation(text=f'<span style="color:red">Observed TVD = {round(obs_stat_o, 2)}</span>',
                   x=0.95 * obs_stat_o, showarrow=False, y=0.09)
#fig.update_layout(yaxis_range=[0, 0.1])
#fig.update_layout(xaxis_range=[0, 0.1])
fig.update_layout(showlegend=False)
fig.update_layout(
    xaxis_title="Generated TVDs"  
)
fig.update_layout(
    width=800,  
    height=600   
)

In [282]:
obs_pivot_month = pivot_table(outage_df, 'MONTH', 'OUTAGE.RESTORATION')

obs_stat_month= obs_pivot_month.diff(axis=1).iloc[:, -1].abs().sum() / 2
obs_stat_month

0.22267850229522704

In [283]:
tvds_month = tvd_perm_test(outage_df, 'MONTH', 'OUTAGE.RESTORATION') 

In [284]:
(tvds_month >= obs_stat_month).mean() 

0.09

In [285]:
fig = px.histogram(pd.DataFrame({'tvds':tvds_month}),histnorm='probability', 
                   title='Empirical Distribution of the TVD')
fig.add_vline(x=obs_stat_month, line_color='red', line_width=2, opacity=1)
fig.add_annotation(text=f'<span style="color:red">Observed TVD = {round(obs_stat_month, 2)}</span>',
                   x=0.9 * obs_stat_month, showarrow=False, y=0.09)
#fig.update_layout(yaxis_range=[0, 0.1])
#fig.update_layout(xaxis_range=[0, 0.1])
fig.update_layout(showlegend=False)
fig.update_layout(
    xaxis_title="Generated TVDs"  
)
fig.update_layout(
    width=800,  
    height=600   
)

In [286]:
o = obs_pivot_month.plot(kind='barh', title='Month by Missingness of Outage Restoration', barmode='group')

o.update_layout(
    width=800,  
    height=600   
)

## Step 4: Hypothesis Testing

Null Hypothesis: The average Impact Score when the Climate is normal the same as the average Impact Score when climate is warm or cold.

Alternate Hypothesis: The average Impact Score when climate is warm or cold is higher than when climate is normal.

Test Statistic: Mean of impact score in abnormal(hot or cold) climate - Mean of impact score in normal climate.

In [287]:
outage_df['CAUSE.CATEGORY'].value_counts()

CAUSE.CATEGORY
severe weather                   763
intentional attack               418
system operability disruption    127
public appeal                     69
equipment failure                 60
fuel supply emergency             51
islanding                         46
Name: count, dtype: int64

In [288]:
weather_df = outage_df[outage_df['CAUSE.CATEGORY']=='severe weather'][['CLIMATE.CATEGORY','IMPACT_SCORE']] # only severe weather
weather_df['CLIMATE.CATEGORY'].value_counts()

CLIMATE.CATEGORY
normal    354
cold      239
warm      166
Name: count, dtype: int64

In [289]:
weather_df['CLIMATE.CATEGORY'] = weather_df['CLIMATE.CATEGORY'].apply(lambda x:'abnormal' if x in ['cold','warm']else x)

In [290]:
means_df = weather_df.groupby('CLIMATE.CATEGORY')['IMPACT_SCORE'].mean()
obs_stat_w = means_df.loc['abnormal'] - means_df.loc['normal']

In [291]:
def mean_perm_test(df, col1,col2):
    np.random.seed(22)
    n_repetitions = 5000
    shuffled = df.copy()
       
    mean_stats = []
    for _ in range(n_repetitions):
        
    
        shuffled[col2] = np.random.permutation(shuffled[col2])
        
        means_df = shuffled.groupby('CLIMATE.CATEGORY')[col2].mean()
        stat = means_df.loc['abnormal'] - means_df.loc['normal']

        mean_stats.append(stat)
    return  mean_stats

In [292]:
mean_stats = mean_perm_test(weather_df,'CLIMATE.CATEGORY','IMPACT_SCORE')

In [293]:
(mean_stats >= obs_stat_w).mean()

0.8266

In [294]:
fig = px.histogram(pd.DataFrame({'mean_stats':mean_stats}),histnorm='probability',
                   title='Empirical Distribution of Difference of Means Statistic')
fig.add_vline(x=obs_stat_w, line_color='red', line_width=2, opacity=1)
fig.add_annotation(text=f'<span style="color:red">Observed TVD = {round(obs_stat_w, 2)}</span>',
                   x=1.8 * obs_stat_w, showarrow=False, y=0.06)
fig.update_layout(yaxis_range=[0, 0.06])
fig.update_layout(showlegend=False)
fig.update_layout(
    xaxis_title="Generated Test Statistics"  
)
fig.update_layout(
    width=800,  
    height=600   
)

We get a very high p-value: 0.8266. So, we fail to reject the null hypothesis. 

Also, I tried doing mean normal - abnormal, resulting in 0.1734 p-value, still failing to reject null hypothesis.

Lastly, I tried doing it with absolute difference of mean, resulting in 0.3586 p-value, still failing to reject null hypothesis.

## Step 5: Framing a Prediction Problem

Prediction Problem: Predict Impact Score of a power outage. Since Impact Score is numerical, this would be a regression model.

In [327]:
pd.set_option('display.max_columns', None)
possible_feature_cols = (['U.S._STATE', 'NERC.REGION', 'CAUSE.CATEGORY', 'ANOMALY.LEVEL',
                          'CLIMATE.REGION','TOTAL.CUSTOMERS','TOTAL.PRICE', 'TOTAL.SALES', 
                          'TOTAL.REALGSP','UTIL.REALGSP','UTIL.CONTRI',
                          'POPULATION', 'POPDEN_URBAN', 'MONTH', 'YEAR','OUTAGE.START'])
outage_focus_df = outage_df[possible_feature_cols + ["IMPACT_SCORE"]].dropna() #only 27 rows lost:
#will not impact model results much since the size is big relative to what was dropped

outage_focus_df

,U.S._STATE,NERC.REGION,CAUSE.CATEGORY,ANOMALY.LEVEL,CLIMATE.REGION,TOTAL.CUSTOMERS,TOTAL.PRICE,TOTAL.SALES,TOTAL.REALGSP,UTIL.REALGSP,UTIL.CONTRI,POPULATION,POPDEN_URBAN,MONTH,YEAR,OUTAGE.START,IMPACT_SCORE
1,Minnesota,MRO,severe weather,-0.3,East North Central,2595696.0,9.28,6562520.0,274182.0,4802.0,1.751391,5348119.0,2279.0,7.0,2011.0,2011-07-01 17:00:00,-0.113518
2,Minnesota,MRO,intentional attack,-0.1,East North Central,2640737.0,9.28,5284231.0,291955.0,5226.0,1.790002,5457125.0,2279.0,5.0,2014.0,2014-05-11 18:38:00,-0.638695
3,Minnesota,MRO,severe weather,-1.5,East North Central,2586905.0,8.15,5222116.0,267895.0,4571.0,1.706266,5310903.0,2279.0,10.0,2010.0,2010-10-26 20:00:00,-0.120676
4,Minnesota,MRO,severe weather,-0.1,East North Central,2606813.0,9.19,5787064.0,277627.0,5364.0,1.932089,5380443.0,2279.0,6.0,2012.0,2012-06-19 04:30:00,-0.179503
5,Minnesota,MRO,severe weather,1.2,East North Central,2673531.0,10.43,5970339.0,292023.0,4873.0,1.668704,5489594.0,2279.0,7.0,2015.0,2015-07-18 02:00:00,0.243088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1527,Idaho,WECC,intentional attack,1.6,Northwest,849763.0,8.05,1657605.0,60911.0,960.0,1.576070,1680026.0,2216.8,3.0,2016.0,2016-03-08 00:00:00,-0.569258
1528,Idaho,WECC,intentional attack,1.6,Northwest,849763.0,8.05,1657605.0,60911.0,960.0,1.576070,1680026.0,2216.8,3.0,2016.0,2016-03-01 13:35:00,-0.522110
1530,North Dakota,MRO,public appeal,-0.9,West North Central,394394.0,7.56,1313678.0,39067.0,934.0,2.390765,685326.0,2192.2,12.0,2011.0,2011-12-06 08:00:00,-0.494073
1532,South Dakota,RFC,islanding,0.5,West North Central,436229.0,7.67,924051.0,36504.0,606.0,1.660092,807067.0,2038.3,8.0,2009.0,2009-08-29 22:54:00,-0.655477


## Step 6: Baseline Model

In [328]:
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [329]:
def results(pipeline, X_train,X_test,y_train,y_test):

    y_pred_train2 = pipeline.predict(X_train)
    y_pred_test2 = pipeline.predict(X_test)

    rmse_train= root_mean_squared_error(y_train, y_pred_train2)
    r2_train = r2_score(y_train, y_pred_train2)

    rmse_test = root_mean_squared_error(y_test, y_pred_test2)
    r2_test = r2_score(y_test, y_pred_test2)

    print(f'RMSE Train: {rmse_train}')
    print(f"r2 Train: {r2_train}")

    print(f'RMSE Test: {rmse_test}')
    print(f"r2 Test: {r2_test}")
    
    return rmse_train,r2_train,rmse_test,r2_test

In [330]:
df_baseline = outage_focus_df.drop(columns = ['TOTAL.SALES','TOTAL.PRICE','UTIL.REALGSP','UTIL.CONTRI','OUTAGE.START'], axis=1)

In [331]:
X = df_baseline.drop(['IMPACT_SCORE'],axis=1)
y = df_baseline['IMPACT_SCORE'] 
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=23)

In [332]:
preproc = ColumnTransformer(
    transformers=[
        ('one-hot', OneHotEncoder(handle_unknown='ignore'), ['U.S._STATE','NERC.REGION','CAUSE.CATEGORY', 'CLIMATE.REGION'])],
    #('selector','passthrough',f)],
    remainder='passthrough',
    force_int_remainder_cols=False,
    verbose_feature_names_out=False
)
pl_random_forest = Pipeline([
    ('prep',preproc),
    ('random_forest', RandomForestRegressor(random_state=42))
])

pl_random_forest.fit(X_train, y_train)
b=results(pl_random_forest,X_train, X_test, y_train, y_test)

RMSE Train: 0.580762509992068
r2 Train: 0.7152538918211958
RMSE Test: 0.8921428877482582
r2 Test: 0.3040027641411822


## Step 7: Final Model

In [333]:

outage_focus_df['HOUR'] = outage_focus_df['OUTAGE.START'].dt.hour
outage_focus_df['DAY'] = outage_focus_df['OUTAGE.START'].dt.day
outage_focus_df = outage_focus_df.drop("OUTAGE.START",axis=1)



outage_focus_df["HOUR_SIN"] = np.sin(2 * np.pi * outage_focus_df["HOUR"] / 24)
outage_focus_df["HOUR_COS"] = np.cos(2 * np.pi * outage_focus_df["HOUR"] / 24)


outage_focus_df["MONTH_SIN"] = np.sin(2 * np.pi * outage_focus_df["MONTH"] / 12)
outage_focus_df["MONTH_COS"] = np.cos(2 * np.pi * outage_focus_df["MONTH"] / 12)

outage_focus_df["DAY_SIN"] = np.sin(2 * np.pi * outage_focus_df["DAY"] / 31)
outage_focus_df["DAY_COS"] = np.cos(2 * np.pi * outage_focus_df["DAY"] / 31)
outage_focus_df = outage_focus_df.drop(['HOUR','DAY','MONTH'],axis=1)

In [334]:
outage_focus_df['CUSTOMERS.DENSITY'] = outage_focus_df['TOTAL.CUSTOMERS'] / outage_focus_df['POPULATION']
outage_focus_df

,U.S._STATE,NERC.REGION,CAUSE.CATEGORY,ANOMALY.LEVEL,CLIMATE.REGION,TOTAL.CUSTOMERS,TOTAL.PRICE,TOTAL.SALES,TOTAL.REALGSP,UTIL.REALGSP,UTIL.CONTRI,POPULATION,POPDEN_URBAN,YEAR,IMPACT_SCORE,HOUR_SIN,HOUR_COS,MONTH_SIN,MONTH_COS,DAY_SIN,DAY_COS,CUSTOMERS.DENSITY
1,Minnesota,MRO,severe weather,-0.3,East North Central,2595696.0,9.28,6562520.0,274182.0,4802.0,1.751391,5348119.0,2279.0,2011.0,-0.113518,-0.965926,-2.588190e-01,-5.000000e-01,-8.660254e-01,0.201299,0.979530,0.485347
2,Minnesota,MRO,intentional attack,-0.1,East North Central,2640737.0,9.28,5284231.0,291955.0,5226.0,1.790002,5457125.0,2279.0,2014.0,-0.638695,-1.000000,-1.836970e-16,5.000000e-01,-8.660254e-01,0.790776,-0.612106,0.483906
3,Minnesota,MRO,severe weather,-1.5,East North Central,2586905.0,8.15,5222116.0,267895.0,4571.0,1.706266,5310903.0,2279.0,2010.0,-0.120676,-0.866025,5.000000e-01,-8.660254e-01,5.000000e-01,-0.848644,0.528964,0.487093
4,Minnesota,MRO,severe weather,-0.1,East North Central,2606813.0,9.19,5787064.0,277627.0,5364.0,1.932089,5380443.0,2279.0,2012.0,-0.179503,0.866025,5.000000e-01,1.224647e-16,-1.000000e+00,-0.651372,-0.758758,0.484498
5,Minnesota,MRO,severe weather,1.2,East North Central,2673531.0,10.43,5970339.0,292023.0,4873.0,1.668704,5489594.0,2279.0,2015.0,0.243088,0.500000,8.660254e-01,-5.000000e-01,-8.660254e-01,-0.485302,-0.874347,0.487018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1527,Idaho,WECC,intentional attack,1.6,Northwest,849763.0,8.05,1657605.0,60911.0,960.0,1.576070,1680026.0,2216.8,2016.0,-0.569258,0.000000,1.000000e+00,1.000000e+00,6.123234e-17,0.998717,-0.050649,0.505803
1528,Idaho,WECC,intentional attack,1.6,Northwest,849763.0,8.05,1657605.0,60911.0,960.0,1.576070,1680026.0,2216.8,2016.0,-0.522110,-0.258819,-9.659258e-01,1.000000e+00,6.123234e-17,0.201299,0.979530,0.505803
1530,North Dakota,MRO,public appeal,-0.9,West North Central,394394.0,7.56,1313678.0,39067.0,934.0,2.390765,685326.0,2192.2,2011.0,-0.494073,0.866025,-5.000000e-01,-2.449294e-16,1.000000e+00,0.937752,0.347305,0.575484
1532,South Dakota,RFC,islanding,0.5,West North Central,436229.0,7.67,924051.0,36504.0,606.0,1.660092,807067.0,2038.3,2009.0,-0.655477,-0.500000,8.660254e-01,-8.660254e-01,-5.000000e-01,-0.394356,0.918958,0.540512


In [335]:
hyperparameters = {
    'random_forest__n_estimators': [100,102,105,150,200], # why? because more trees means more diversity(in usage of features and predictions) so could improve predictions
    'random_forest__max_depth': [10,11,12, 13, 14,15, 20, None] # because of over fitting and best depth for predictions
# note that i tried other hyperparameters but did not give me good results so just left them as default
}


In [336]:
X = outage_focus_df.drop(['IMPACT_SCORE'],axis=1)
y = outage_focus_df['IMPACT_SCORE']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=23)

In [337]:

# Takes a few seconds to run – how many trees are being trained?
grids = GridSearchCV(
    pl_random_forest,
    n_jobs=-1, # Use multiple processors to parallelize
    param_grid=hyperparameters,
    return_train_score=True
)
grids.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('prep',
                                        ColumnTransformer(force_int_remainder_cols=False,
                                                          remainder='passthrough',
                                                          transformers=[('one-hot',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['U.S._STATE',
                                                                          'NERC.REGION',
                                                                          'CAUSE.CATEGORY',
                                                                          'CLIMATE.REGION'])],
                                                          verbose_feature_names_out=False)),
                                       ('random_forest',
                                        RandomForestRegressor(random_state=42))]),
             n_jobs=-1,
             param_grid={'random_forest__max_depth': [10, 11, 12, 13, 14, 15,
                                                      20, None],
                         'random_forest__n_estimators': [100, 102, 105, 150,
                                                         200]},
             return_train_score=True)

In [340]:
grids.best_params_

{'random_forest__max_depth': 10, 'random_forest__n_estimators': 102}

In [341]:
s=results(grids,X_train, X_test, y_train, y_test)

RMSE Train: 0.5160649382563459
r2 Train: 0.7751621842287234
RMSE Test: 0.8069550707914643
r2 Test: 0.4305738735545015


## Step 8: Fairness Analysis

In [309]:
y_pred = grids.predict(X_test)
cutoff =X['POPULATION'].quantile(0.75)
results = X_test.copy()
results['prediction'] = y_pred
results['actual'] = y_test
results['is_highly_populated'] = (results['POPULATION'] > cutoff).replace({True: 'yes', False: 'no'})
results

,U.S._STATE,NERC.REGION,CAUSE.CATEGORY,ANOMALY.LEVEL,CLIMATE.REGION,TOTAL.CUSTOMERS,TOTAL.PRICE,TOTAL.SALES,TOTAL.REALGSP,UTIL.REALGSP,UTIL.CONTRI,POPULATION,POPDEN_URBAN,YEAR,HOUR_SIN,HOUR_COS,MONTH_SIN,MONTH_COS,DAY_SIN,DAY_COS,CUSTOMERS.DENSITY,prediction,actual,is_highly_populated
1023,Louisiana,SERC,equipment failure,-0.2,South,2311899.0,7.92,6406309.0,205576.0,3907.0,1.900514,4627491.0,1685.8,2013.0,-0.965926,-2.588190e-01,0.500000,-0.866025,-0.790776,-0.612106,0.499601,-0.448359,-0.589160,no
89,Michigan,RFC,severe weather,-0.4,East North Central,4829741.0,8.76,10002112.0,420411.0,8589.0,2.043001,10001284.0,2034.1,2007.0,-0.965926,2.588190e-01,-0.500000,-0.866025,0.848644,0.528964,0.482912,0.107270,0.033946,no
860,Missouri,SPP,severe weather,-0.4,Central,3072941.0,8.56,6132823.0,249820.0,4845.0,1.939396,6010587.0,2053.5,2011.0,-0.965926,-2.588190e-01,0.500000,-0.866025,-0.968077,-0.250653,0.511255,-0.080322,0.888024,no
554,Maryland,RFC,severe weather,0.3,Northeast,2468101.0,11.31,4452722.0,317123.0,6643.0,2.094771,5890740.0,2511.4,2012.0,-0.965926,-2.588190e-01,-0.866025,0.500000,-0.394356,0.918958,0.418980,0.319136,0.246055,no
26,Tennessee,SERC,equipment failure,0.5,Central,3153916.0,8.59,8950358.0,247321.0,1598.0,0.646124,6306019.0,1450.3,2009.0,0.965926,2.588190e-01,-0.500000,-0.866025,-0.724793,0.688967,0.500144,-0.307226,-0.287312,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529,Maryland,RFC,intentional attack,-1.0,Northeast,2461923.0,12.34,5155310.0,315930.0,6549.0,2.072928,5844171.0,2511.4,2011.0,-0.500000,-8.660254e-01,0.866025,0.500000,0.571268,0.820763,0.421261,-0.570237,-0.564433,no
186,Texas,TRE,public appeal,-0.3,South,11084145.0,9.34,38540463.0,1246833.0,28045.0,2.249299,25654464.0,2435.3,2011.0,0.258819,-9.659258e-01,-0.500000,-0.866025,0.299363,-0.954139,0.432055,-0.473854,-0.573899,yes
1422,Oklahoma,SPP,severe weather,2.2,South,2020012.0,7.25,4231978.0,181045.0,4198.0,2.318761,3911338.0,1901.7,2015.0,1.000000,6.123234e-17,-0.500000,0.866025,-0.571268,0.820763,0.516450,0.354804,-0.155746,no
810,South Carolina,SERC,severe weather,-0.5,Southeast,2511089.0,9.68,6513433.0,173785.0,4318.0,2.484679,4829160.0,1288.1,2014.0,0.258819,-9.659258e-01,0.866025,0.500000,0.651372,-0.758758,0.519985,0.551344,0.162898,no


In [310]:
compute_rmse = lambda x: root_mean_squared_error(x['actual'], x['prediction'])

In [311]:
(
    results
    .groupby('is_highly_populated')
    [['actual', 'prediction']]
    .apply(compute_rmse)
    .rename('rmse').diff()
)

is_highly_populated
no          NaN
yes    0.029629
Name: rmse, dtype: float64

### Is this difference in RMSE significant?

Let's run a **permutation test** to see if the difference in accuracy is significant.
- **Null Hypothesis**: Our model is fair. The RMSE is the same for both highly populated and not highly populated states, and any differences is due to chance.
- **Alternative Hypothesis**: Our model is unfair. The RMSE is higher for highly populated states.
- Test statistic: Difference in accuracy (young minus old).
- Significance level: 0.01.

In [312]:

obs = (
    results
    .groupby('is_highly_populated')
    [['actual', 'prediction']]
    .apply(compute_rmse)
    .rename('rmse').diff().iloc[-1]
)
obs

0.02962867617075171

In [313]:
np.random.seed(21)
diff_in_acc = []
for _ in range(1000):
    s = (
        results[['is_highly_populated', 'prediction', 'actual']]
        .assign(is_highly_populated=np.random.permutation(results['is_highly_populated']))
        .groupby('is_highly_populated')
        [['actual', 'prediction']]
        .apply(compute_rmse)
        .diff()
        .iloc[-1]
    )
    
    diff_in_acc.append(np.abs(s))

In [314]:
p_value = (diff_in_acc >= obs).mean()
p_value

0.971

In [315]:
fig = px.histogram(pd.DataFrame({'diff_means':diff_in_acc}),histnorm='probability',
                   title='Empirical Distribution of the difference in RMSE')
fig.add_vline(x=obs, line_color='red', line_width=2, opacity=1)
fig.add_annotation(text=f'<span style="color:red">Observed TVD = {round(obs, 2)}</span>',
                   x=4.2 * obs, showarrow=False, y=0.14)
fig.update_layout(showlegend=False)
fig.update_layout(
    xaxis_title="Generated Difference in RMSE"  
)
fig.update_layout(
    width=800,  
    height=600   
)